# LLM Client (大模型客户端) 实战

本笔记本演示了如何封装一个通用的 LLM 客户端，用于调用兼容 OpenAI 接口的大模型服务（如 GitHub Models, DeepSeek, OpenAI 等）。

### 核心功能：
1. **统一接口**：通过 `HelloAgentsLLM` 类封装复杂的调用细节。
2. **环境变量管理**：使用 `.env` 文件隔离密钥安全。
3. **流式输出**：实时打印模型生成的内容，提升用户体验。
4. **C/C++ 类比**：将 Python 类和方法与 C++ 概念对照理解。

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

# 1. 导入库与环境配置
# load_dotenv() 类比 C++ 的 LoadConfig() 函数，从磁盘读取变量并注入系统环境。
# 默认会寻找项目根目录下的 .env 文件。
load_dotenv(dotenv_path="../../.env")

# 检查环境变量是否加载成功
model_id = os.getenv("LLM_MODEL_ID")
print(f"当前配置的模型: {model_id}")

当前配置的模型: gpt-4o-mini


In [4]:
class HelloAgentsLLM:
    """
    为本书 "Hello Agents" 定制的LLM客户端。
    类比 C++ 的 Wrapper Class，将底层的 OpenAI SDK 封装成适合本项目使用的接口。
    """
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        """
        构造函数。
        """
        # 这里的 or 运算逻辑类比 C++ 的三元运算符或默认参数处理。
        self.model = model or os.getenv("LLM_MODEL_ID")
        apiKey = apiKey or os.getenv("LLM_API_KEY")
        baseUrl = baseUrl or os.getenv("LLM_BASE_URL")
        timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))
        
        if not all([self.model, apiKey, baseUrl]):
            raise ValueError("模型ID、API密钥和服务地址必须被提供或在.env文件中定义。")

        # 实例化 OpenAI 客户端，类比 C++ 的智能指针或 Resource Manager
        self.client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        """
        核心思考方法。
        """
        print(f"🧠 正在调用 {self.model} 模型...")
        try:
            # stream=True 开启流式响应，类比 C++ 的 Callback 或异步流。
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True,
            )
            
            print("✅ 大语言模型响应成功:")
            collected_content = []
            # 迭代 chunk 类比 C++ 的 while(stream.read())
            for chunk in response:
                if chunk.choices:
                    content = chunk.choices[0].delta.content or ""
                    print(content, end="", flush=True) # flush=True 确保字符立即显示在屏幕上
                    collected_content.append(content)
            print() 
            return "".join(collected_content)

        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return None

In [5]:
# 2. 客户端使用示例
# 这里演示如何实例化并调用，类比 C++ 的 main 函数测试部分。

try:
    # 自动从 .env 载入配置
    llmClient = HelloAgentsLLM()
    
    # 构造对话消息，类比 Vector<Map<String, String>>
    exampleMessages = [
        {"role": "system", "content": "You are a helpful assistant that writes Python code."},
        {"role": "user", "content": "写一个快速排序算法"}
    ]
    
    print("--- 调用LLM ---")
    responseText = llmClient.think(exampleMessages)
    
    if responseText:
        print("\n--- 完整模型响应预览 ---")
        # 打印生成的字符总数，类比 string.length()
        print(f"生成的文本长度: {len(responseText)}")

except ValueError as e:
    print(f"初始化失败: {e}")
except Exception as e:
    print(f"发生未知错误: {e}")

--- 调用LLM ---
🧠 正在调用 gpt-4o-mini 模型...
✅ 大语言模型响应成功:
快速排序（Quick Sort）是一种高效的排序算法，采用分治法（Divide and Conquer）策略。以下是一个用 Python 实现的快速排序算法的示例代码：

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]  # 选择中间的元素作为基准
        left = [x for x in arr if x < pivot]  # 小于基准的元素
        middle = [x for x in arr if x == pivot]  # 等于基准的元素
        right = [x for x in arr if x > pivot]  # 大于基准的元素
        return quick_sort(left) + middle + quick_sort(right)

# 示例用法
if __name__ == "__main__":
    arr = [3, 6, 8, 10, 1, 2, 1]
    sorted_arr = quick_sort(arr)
    print("排序后的数组:", sorted_arr)
```

### 代码说明：
1. **基准选择**：选择数组中间的元素作为基准（pivot）。
2. **分区**：将数组分为三个部分：
   - 小于基准的元素（left）
   - 等于基准的元素（middle）
   - 大于基准的元素（right）
3. **递归排序**：对小于和大于基准的部分递归调用 `quick_sort` 函数。
4. **合并结果**：将排序后的左部分、基准部分和右部分合并成一个新的排序数组。

### 示例输出：
运行上述代码将输出：
```
排序后的数组: [1, 1, 2, 3, 6, 8, 10]
```

你可以根据需要修改输入数组 `arr` 来测试不同的情况。

--- 完整模型响应预览 ---
生成的文本长度: 893
